## Lab (Final) Report - Predicting Lane Matchups in League of Legends

## Due 28 November 2016 (Monday, 11:59pm)


### Abstract:

The purpose of this project is to create a usable application for League of Legends that obtains two opposing players' data from previous games and uses this information to predict who would defeat the other. From the midterm report, I focused on time series regression, as it seems intuitive that past results will help predict the desired classification. Since then, I went towards the data collection and classification using a standard MLP. I developed several methodologies that helped progress my level of understanding of my problem as well as just neural networks in general. Although I did not obtain high test accuracies this time around, I believe that I've outlined some future modifications that can result in better results. The initial modeling, without ever the introduction of neural networks, has proven to be an essential part of the project.

### Improvement:

Since the last report, I applied a classification model using an MLP towards the heart of my project: predicting lane matchups in League of Legends. I went through many steps to try to get this to work, but unfortunately I did not plan enough time to test out several methods. I do, however, discuss what I could test in the future to further improve my results. For my current findings, I was able to get ~80% accuracy on training and validation, however, when using my testing set, I only got to 58%, even after regularization and hyperparameter optimization. I went through a lot of testing and gained intuition on how several hyperparameters affect the results, but I believe that the way I set up the problem would have never achieved the desired results. More is discussed in later sections. 

One big finding I found, however, is that the features I chose are predictive of what I want with the use of only one player's data (rather than collectively using several players). I found that I got an accuracy of 87.55% and 85.94% training and validation accuracy (effectively removing overfitting) and a testing accuracy of 86.66%. This puts me at ease knowing the features I chose can still work. 

One more thing I did not get to work on was the implementation of the LSTM. I have kept the discussion on this report because I feel it can be a powerful tool to fill up the gap in my study.



### Introduction:

With the release of the Riot API, many 3rd party applications have been created to cater to League of Legends players that provide enemy statistics, histories, and pregame setups [6]. It has been my goal to create my own software to provide me with statistics that matter to me: e.g. will I be able to defeat the player I am up against? My project should do just that; using past data provided by Riot, I want to be able to create a predictor that gives me insight as to how the match will play out. Because players tend to learn as they play more and more games, it seems natural to implement a model that works for sequences of times.

Some background on the statistics I will use is important. They are: Champions, KDA, CS (Creep Score), Exp. So the game of League of Legends contains many characters, or champions. Each champion has distinct characteristics that let them fill a specific role in every game. Thus, inherently, I believe that an important part to the matchup is what two champions opposing players choose to play. KDA stands for Kill+Assists:Deaths Ratio. This measures just how well you "fight" in League; when you get a kill or assist on an enemy, the KDA goes up, while if you die, then the KDA will go down. Thus, if we can collect a player's KDA on a specific champion, then we should get an indication of how they play. 

Creep score (CS), describes how many minions a player kills. Both teams spawn minions every 30 seconds and killing them provides a small amount of gold. [10] Thus, without fighting, a player can effectively gain more gold than the opponent as long as they get more CS. Playstyle is important! Some players like to be aggressive in lane, adding risk to both parties, while others stay passive and win lane without making any kills (by accruing a large amount of CS or taking objectives). The numerics provided by the API help visualize these concepts a neural network could understand. 

When we do learn over time, we use our past experiences to determine a new decision in the present; analogously, if we have data points at past time indeces, we could use those to predict the very next data point, if given evidence that they exhibit a time series pattern. One such example would be in the world of finance; patterns that repeat over certain periods of time occur more often than not, and being able to predict those with neural networks can be very useful [7]. Thus, to begin in applying it to my own problem, I have to understand the concepts of time series regression using a standard multilayer perceptron on simple periodic functions.

After doing so, then I can enter the realm of recurrent neural networks. What makes a recurrent neural network different from an MLP is that the hidden layer contains loops. [2] I.e., we assume that each input at a certain time produces an output in a hidden neuron that increases the predictive ability of the network in future time indeces:


<img src="./rnn.jpg">

This concept encapsulates the "memory" aspect of time sequence problems. Because the structure diverges from an MLP, it was necessary to create a new backpropagation algorithm for RNN's: Backpropagation through time (BPTT). However, due to the nature of backpropagation, if we include a large time step window, the "memory" from an early time index will have little effect on the error (due to many operations of multiplication), thus training would take much too long to optimize results [9]. 

This is where Long Short-term memory (LSTM) networks come into play: instead of having loops inside a hidden layer, the structure of neurons in a hidden layer is fundamentally changed. The neurons are replaced with LSTM cells, that contain 3 components: an input gate, output gate, and forget gate. This "forget" gate, powered by a sigmoid layer determines whether or not certain pieces of information should be kept for future time steps. In doing so, even through large time steps, data that has been deemed important will not be replaced by lesser information during the process [8]. The following figure gives a visual of the structure:

<img src="./lstm_structure.png">

### Methodology

*** Data Collection ***

To collect the data I used in my experiments, I utilized the Riot API to make requests on 10 players. I had 3 main criteria when collecting matches across each player:

* The player I'm looking for is playing **TOP LANE**
* The match was in **Season 6**
* The match was either in a **Solo or Team Ranked** Queue.

If any matches filled these criteria, then I cached it locally so that I would not have to search for it again (this is important since Riot API implements an API request limit).

Afterwards, for every match for a given player, I found their opponent and I looked up the last 25 matches that filled this criterion:

* The opponent is playing the **same champion** as the original game

With all the collected data, I was ready to perform classification. I will discuss several methods and whether or not I continued with it. I did, however, use these features for the different models (each feature was doubled since I took the same statistic for both players in a game):

* Champion Id
* KDA
* Champion Winrate
* First blood rate - how often a player participates in the first kill of the game
* Summoner Spells
* CS/min (0-10 and 10-20 minutes) - how many minions a player kills per minute with a time frame
* Exp/min (0-10 and 10-20 minutes) - how much experience points a player gains per minute in a time frame.

I chose to take the first 20 minutes to define the "laning" phase of League. Thus, to determine which player won lane for a specific game, I took:

* **Gold at 20 minutes**

Gold provides an objective way to choose a winner, since if someone got more gold by 20 minutes, they must have performed better.

The following are the different methods I went through [I used Keras in every model] [1b]:

*** Model A: Features as averages for Binary Classification***

I initially believed that if I took the average of stats such as KDA, CS/min and XP/min over the last 25 games for both players on a specific champion, I could get some sense of "past results." This assumption is flawed, however, since I lose specific game data by averaging each result. Nevertheless I took this process:

* I standardized the feature vectors (after calculating the averages I desired) over each feature using the z-score formula
* Created a MLP with the following hyperparameters:
    * 22 Input Neurons
    * 2 Hidden Layers (with dropout and ReLU Activation)
    * 1 Output Neuron (with Sigmoid Activation)
    * Binary Cross Entropy loss function 
    * Adadelta optimizer
    * Full batch training
    * 1000 epochs

I found that on the training set without regularization, I overfit by a lot, but as I tried to fine tune regularization with the other hyperparameters, I found that the model would eventually converge to a certain accuracy that was too low to be considered good.

*** Model B: Regular Features, using regression to predict future feature values ***

Since using a moving average did not get good results, I decided to switch to this new model, which makes some sense initially, but if taken at a closer look, it was evident why it is currently unsuccessful.

The process I went through for regression (I did not implement feature prediction with the regression because regression was unsuccessful):  
* Standardized raw feature data
* Take known data that could be obtained before a League of Legends game and use as inputs for MLP (12 features)
* Take unknown data to be collected in the future and use as outputs (10 features)
* Thus, my MLP had the following structure:
    * 12 input neurons
    * 2 Hidden Layers (with 128 neurons and 0.6 dropout and ReLU Activation each)
    * 10 Output Neurons (with Linear Activation)
    * MSE loss function
    * rmsprop optimizer
    * Full batch training
    * 1000 epochs

As seen later in the results, the errors on just the training set were too high for me to continue going, so I made the final model:

*** Model C: Regular Features, assuming unknown values are given ***

This is synonymous to asking ***"How predictive are the features I chose?"*** This at least gave me a way to confirm some methodologies to lead me on future work for this project. 

Using only one player's games, I was able to get solid results (only ~400 matches)

For training and validation, I used the first 80% of this dataset, (using 80% of it for training and 20% for validation) and the last 20% was used as testing.
I used a standard MLP with the following hyperparameters:

* 22 input neurons (contains all the features mentioned earlier)
* 1 Hidden Layer (with 32 neurons and 0.5 dropout and ReLU Activation with L2 Regularization)
* 1 Output Neuron (with Sigmoid Activation)
* Binary Cross Entropy loss function
* adadelta optimizer
* Full batch training
* 1000 epochs

In addition, I applied a logistic regression model with feature selection (12 chosen out of 22) to get a little more insight on why they are predicting decently.
   

### Results and Discussion

*** Model A ***
The results of this model is shown in two figures: **I** without regularization and **II** with regularization.

**I)**

<img src="average_overfitting.png">

**II)**

<img src="average_regularization.png">

You can immediately see that in **Figure I** that the data overfits to a great extent, even leaving the validation accuracy in the 50%s. If I let the epoch length to be longer, the training model will almost certainly converge to 100%, but it is no use if the results cannot be generalized.

In **Figure II**, this is the result of many tinkerings around hyperparameters, in order to both reduce overfitting and increase training and validation accuracy. The results were to no avail, and in fact, when I applied a trained model to my testing set, I could only get up to **59%** accuracy, which is hardly better than random. 

This is probably due to the fact that I used average statistics rather than the raw collected ones from the match histories, since by taking these averages, I lose fine details that could potentially help classify the dataset. (This is eventually seen in **Model C**)

*** Model B ***

The figure below is a plot taken from training my MLP using validation:

<img src="regression.png">

As seen, both the training and validation sets converged to around 0.9 MSE, but because my values were standardized, this was no good at all. Initially, this had a good premise, but I believe the known features chosen were just not related to the unknowns. As a recap these were the known features (both players' counterparts per game):

**Known**
* Summoner spells
* First blood rate
* Champion
* Champion # games
* Champion win rate

**Unknown**
* KDA
* CS10, CS20
* XP10, XP20

I assert that this does not get any traction for regression, because the unknowns are in-game stats while the known values are just past statistics that do not have to correlate with the statistics I desired. There is also no intuition in connecting ***how*** a person performs with a champion with their **future** stats dealing with micro mechanics. It is evident that this is true just from seeing the results from the regression. I talk about future ideas in the last section of this report.


*** Model C *** 

I plotted the accuracy of prediction of my MLP in the following figure:

<img src="feature_test.png">

The final accuracies were:

* **Training**: 82.61%
* **Validation**: 87.50%
* **Testing**: 82.66%

Very little overfitting exists with a much greater accuracy then ** Model A **. This is strong evidence that the original features I chose to use can classify how a lane matchup will go very well. To get some more insight, I got the following results using scikit-learn's logistic regression model (with 12 feature selection):

             precision    

          0       0.86   
          
          1       0.82  

    avg / total   0.84 
    
    
Chosen Features with Highest Rank:

* Player 1 **Spell 1**
* Player 1 **Champion**
* Player 1 **First blood rate**
* Player 1 **CS10, CS20, XP20**
* Player 2 **Champion win rate**
* Player 2 **Spell 1 and 2**
* Player 2 **First blood rate**
* Player 2 **CS20, XP10, XP20**

These features were most able to model the total variance as each data point was categorized into either a won/lost lane. Of course, most the unknown values had been chosen, but it looks like first blood rate is very prominent in this analysis; maybe suggesting that playing aggressively in lane leads to a more definite win/lose lane scenario.

### Conclusions and Future Work

So by going through these 3 models, I have made some conclusions that can help me as I continue to improve this project. In model A, it gave a prime example of why average stats should not be used, and I also gained some insights on hyperparameter modification. I have realized that optimizing a neural network is not 'clear cut,' it takes a strong foundation of concepts, perseverance and a little luck to obtain a potential structure. In model B, it gave me a chance to explore a possible solution by using regression before classification to obtain future values. I think I've come up with a plausible solution to address this and hopefully obtain better results (which I will describe in the next paragraph). And lastly, Model C gave me some hope in that these features I have chosen are representative of the problem at hand. Hopefully I find a suitable model in the future that will do what I intended to accomplish.

In the future, I want to start my model testing by going back to the regression concept; instead of using the known features to predict unknown features, I could instead limit myself to these unknowns. I.e., if given a history of values for a certain feature, write a regression model that predicts the future value of this feature. This directly ties into my other work with time series regression, and maybe using an LSTM could get some unexpected results. 
In addition, I did collect ~10 players for my training set and then myself as the test set, but I could have also restricted myself to one player per neural network. This actually makes more sense after going through the modeling process, because one neural network will not be able to describe more than 1 person; each player has their own tendencies that can have different results. After that, I could maybe branch out to other classification algorithms that have been used to see if they can more efficiently find a relationship in the data. My eventual goal for this is to have something fully functional that I can create an online service that would not only be used by me, but also all my friends and even any League player in any region, because I know this can be a tool that can improve players' results significantly.

### References ###

[1] Martín Abadi, Ashish Agarwal, Paul Barham, Eugene Brevdo,
Zhifeng Chen, Craig Citro, Greg S. Corrado, Andy Davis,
Jeffrey Dean, Matthieu Devin, Sanjay Ghemawat, Ian Goodfellow,
Andrew Harp, Geoffrey Irving, Michael Isard, Rafal Jozefowicz, Yangqing Jia,
Lukasz Kaiser, Manjunath Kudlur, Josh Levenberg, Dan Mané, Mike Schuster,
Rajat Monga, Sherry Moore, Derek Murray, Chris Olah, Jonathon Shlens,
Benoit Steiner, Ilya Sutskever, Kunal Talwar, Paul Tucker,
Vincent Vanhoucke, Vijay Vasudevan, Fernanda Viégas,
Oriol Vinyals, Pete Warden, Martin Wattenberg, Martin Wicke,
Yuan Yu, and Xiaoqiang Zheng.
TensorFlow: Large-scale machine learning on heterogeneous systems,
2015. Software available from tensorflow.org.

[2] Britz, Denny. "Recurrent Neural Networks Tutorial, Part 1 – Introduction ..." WildML. Google Brain http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

[3] Brownlee, Jason. "Time Series Prediction with LSTM Recurrent Neural Networks ..." Machine Learning Mastery. http://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

[4] Chollet, Francois, "Keras." Github. 2015. 
https://github.com/fchollet/keras

[5] Huang, Thomas, David Kim, and Gregory Leung. "League of Legends Win Predictor."
http://thomasythuang.github.io/League-Predictor/

[6] Kica, Artian. Analysis of Data Gathered from League of Legends and the Riot Games API. Diss. Worcester Polytechnic Institute, 2016.

[7] Kaastra, Iebeling, and Milton Boyd. "Designing a neural network for forecasting financial and economic time series." Neurocomputing 10.3 (1996): 215-236.

[8] Olah, Christopher. "Understanding LSTM Networks — Colah’s Blog." http://colah.github.io/posts/2015-08-Understanding-LSTMs/

[9] Pascanu, Razvan, Tomas Mikolov, and Yoshua Bengio. "On the difficulty of training recurrent neural networks." ICML (3) 28 (2013): 1310-1318.

[10] http://leagueoflegends.wikia.com/wiki/Minion